# Google Drive

In [2]:
####################################
#
#  ADD THIS TO EVERY COLAB FILE!
#
####################################

from google.colab import drive
drive.mount('/content/drive')

import drive.Shareddrives.GPTJ.project.settings as settings

PATH_PROJECT = settings.PATH_PROJECT
PATH_DATA = settings.PATH_DATA

Mounted at /content/drive


In [3]:
! cd $PATH_PROJECT && pip install -q -r requirements.txt

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 143 kB 4.8 MB/s 
     |████████████████████████████████| 55.9 MB 234 kB/s 
     |████████████████████████████████| 85 kB 4.5 MB/s 
     |████████████████████████████████| 72 kB 1.4 MB/s 
     |████████████████████████████████| 7.6 MB 42.8 MB/s 
     |████████████████████████████████| 182 kB 60.2 MB/s 
     |████████████████████████████████| 1.3 MB 59.8 MB/s 
     |████████████████████████████████| 451 kB 3.2 MB/s 
     |████████████████████████████████| 212 kB 62.8 MB/s 
     |████████████████████████████████| 115 kB 63.2 MB/s 
     |████████████████████████████████| 127 kB 62.0 MB/s 


# Processamento dos arquivos

Filtro de opcodes -> mantemos apenas as palavras que são opcodes ou adjacentes a um opcode

In [ ]:
! cd $PATH_PROJECT && python main.py --process

Namespace(batch=None, cache=False, check=False, complete=False, epochs=None, label=None, limit=None, metrics=False, model=None, multi=False, path=None, process=True, test=False, train=False)
pid: 399
Traceback (most recent call last):
  File "main.py", line 240, in <module>
    main(args)
  File "main.py", line 40, in main
    proc.sanity_check()
  File "/content/drive/Shareddrives/GPTJ/project/processor.py", line 161, in sanity_check
    with open(new_path, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/Shareddrives/GPTJ/data/kaggle/proc-1/1/01kcPWA9K2BOxQeS5Rju.asm'


# Cache local do Dataset

Como o dataset tem um tamanho grande e está no Google Drive distribuído em vários arquivos, queremos carregar todos em um único arquivo já tokenizado pelo *tokenizer* da GPT.

Para evitar problemas de sincronização, vamos copiar os arquivos para /content/tmp/, depois copiar de volta os arquivos resultantes.

In [ ]:
! mkdir -p "/content/tmp/"
! for i in {0..8}; do echo "model $i" && time cp -r /content/drive/Shareddrives/GPTJ/data/kaggle/proc-1/$i /content/tmp/; done

model 0

real	1m7.935s
user	0m0.029s
sys	0m2.559s
model 1

real	1m49.951s
user	0m0.047s
sys	0m2.417s
model 2

real	1m27.989s
user	0m0.057s
sys	0m1.067s
model 3

real	0m9.085s
user	0m0.013s
sys	0m0.185s
model 4

real	0m32.889s
user	0m0.007s
sys	0m0.017s
model 5

real	0m16.938s
user	0m0.008s
sys	0m0.347s
model 6

real	0m9.975s
user	0m0.008s
sys	0m0.185s
model 7

real	0m43.156s
user	0m0.027s
sys	0m0.627s
model 8

real	0m43.172s
user	0m0.018s
sys	0m0.733s


In [ ]:

# ! ls "$PATH_DATA/kaggle/proc-1/$LABEL/" | wc -l
# ! ls "/content/tmp/$LABEL/" | wc -l

! for i in {0..8}; do echo "model $i" && du -sh "/content/tmp/$i" && ls "/content/tmp/$i" | wc -l; done

model 0
1.3G	/content/tmp/0
1541
model 1
989M	/content/tmp/1
2478
model 2
76M	/content/tmp/2
2942
model 3
16M	/content/tmp/3
475
model 4
1.8M	/content/tmp/4
42
model 5
67M	/content/tmp/5
751
model 6
28M	/content/tmp/6
398
model 7
141M	/content/tmp/7
1228
model 8
137M	/content/tmp/8
1013


In [ ]:
! cd $PATH_PROJECT && python main.py --cache --model='gpt2' --limit='10240' --fold='1' --path='/content/tmp/'

Namespace(batch=None, cache=True, check=False, complete=False, epochs=None, fold='1', label=None, limit='10240', metrics=False, model='gpt2', multi=False, path='/content/tmp/', process=False, test=False, train=False)
pid: 10827
Args: limit 10240
Args: epochs 2
Args: batch 160
Args: labels ['0', '1', '2', '3', '4', '5', '6', '7', '8']
Args: path /content/tmp/
Args: model_name gpt2
Args: multi False
Args: fold 1
Model: name gpt2
Model: label '0'
Model: num_labels 2
Model: using saved tokenizer...
-----
label 0
-----
label 1
-----
label 2
-----
label 3
-----
label 4
-----
label 5
-----
label 6
-----
label 7
-----
label 8
Dataset: label 0 - train - in model as torch.tensor(0)
Dataset: label 0 - train - creating from /content/tmp/0 ...
Dataset: label 0 - train - 1233 files
Token indices sequence length is longer than the specified maximum sequence length for this model (3013 > 1024). Running this sequence through the model will result in indexing errors
Dataset: label 0 - train - 0%
Dataset

In [ ]:
# copy files back
! time cp /content/tmp/*.csv "/content/drive/Shareddrives/GPTJ/data/kaggle/proc-1/"


real	0m54.296s
user	0m0.120s
sys	0m8.505s


# Cache

In [4]:
# cache dataset
! cd $PATH_PROJECT && python main.py --cache --model='gpt2' --limit='102400' --fold='9'
! cd $PATH_PROJECT && python main.py --cache --model='gpt2' --limit='102400' --fold='10'

Namespace(batch=None, cache=True, check=False, complete=False, epochs=None, fold='9', label=None, limit='102400', metrics=False, model='gpt2', multi=False, path=None, process=False, test=False, train=False)
pid: 387
Args: limit 102400
Args: epochs 2
Args: batch 160
Args: labels ['0', '1', '2', '3', '4', '5', '6', '7', '8']
Args: path /content/drive/Shareddrives/GPTJ/data/kaggle/proc-1
Args: model_name gpt2
Args: multi False
Args: fold 9
Model: name gpt2
Model: label '0'
Model: num_labels 2
Model: downloading tokenizer...
Downloading: 100% 1.04M/1.04M [00:00<00:00, 9.79MB/s]
Downloading: 100% 456k/456k [00:00<00:00, 4.80MB/s]
Downloading: 100% 665/665 [00:00<00:00, 542kB/s]
Model: saving tokenizer...
Model: tokenizer saved!
-----
label 0
-----
label 1
-----
label 2
-----
label 3
-----
label 4
-----
label 5
-----
label 6
-----
label 7
-----
label 8
Dataset: label 0 - train - in model as torch.tensor(0)
Dataset: label 0 - train - creating from /content/drive/Shareddrives/GPTJ/data/kaggle/

# Treino

In [ ]:
! cd $PATH_PROJECT && python main.py --train --multi --model='gpt2' --limit='102400' --epochs='2' --batch='160'

Namespace(batch='160', cache=False, check=False, complete=False, epochs='2', label=None, limit='102400', metrics=False, model='gpt2', multi=True, path=None, process=False, test=False, train=True)
pid: 351
Args: limit 102400
Args: epochs 2
Args: batch 160
Args: labels ['all']
Args: path /content/drive/Shareddrives/GPTJ/data/kaggle/proc-1
Args: model_name gpt2
Args: multi True
START all
Model: name gpt2
Model: label 'all'
Model: num_labels 9
Model: using saved tokenizer...
END all
{}
0:00:01


# Test Binary

In [ ]:
! cd $PATH_PROJECT && python main.py --test --model='gpt2' --limit='1024' --batch='16'

Namespace(batch='16', cache=False, check=False, complete=False, epochs=None, label=None, limit='1024', metrics=False, model='gpt2', multi=False, path=None, process=False, test=True, train=False)
pid: 17025
Args: limit 1024
Args: epochs 2
Args: batch 16
Args: labels ['0', '1', '2', '3', '4', '5', '6', '7', '8']
Args: path /content/drive/Shareddrives/GPTJ/data/kaggle/proc-1
Args: model_name gpt2
Args: multi False
Model: name gpt2
Model: label '0'
Model: num_labels 2
Model: using saved tokenizer...
Dataset: label 0 - test - in model as torch.tensor(0)
Dataset: label 0 - test - loading from /content/drive/Shareddrives/GPTJ/data/kaggle/proc-1/0.limit-1024.chunk-32.test.csv ...
Dataset: label 0 - test - 4922 chunks - 0.04 Gb
Dataset: label 1 - test - in model as torch.tensor(1)
Dataset: label 1 - test - loading from /content/drive/Shareddrives/GPTJ/data/kaggle/proc-1/1.limit-1024.chunk-32.test.csv ...
Dataset: label 1 - test - 7872 chunks - 0.06 Gb
Dataset: label 2 - test - in model as torch

In [ ]:
! cd $PATH_PROJECT && python main.py --metrics --model='gpt2' --limit='512' --batch='16'

# Test Multi

In [ ]:
! cd $PATH_PROJECT && python main.py --test --multi --model='gpt2' --limit='102400' --epochs='2' --batch='160'

# Metrics Multi

In [ ]:

! cd $PATH_PROJECT && python main.py --metrics --multi --model='gpt2' --limit='10240' --epochs='1' --batch='160'
! cd $PATH_PROJECT && python main.py --metrics --multi --model='gpt2' --limit='10240' --epochs='2' --batch='160'
! cd $PATH_PROJECT && python main.py --metrics --multi --model='gpt2' --limit='10240' --epochs='3' --batch='160'


! cd $PATH_PROJECT && python main.py --metrics --multi --model='gpt2' --limit='102400' --epochs='1' --batch='160'
! cd $PATH_PROJECT && python main.py --metrics --multi --model='gpt2' --limit='102400' --epochs='2' --batch='160'

Namespace(batch='160', cache=False, check=False, complete=False, epochs='1', label=None, limit='10240', metrics=True, model='gpt2', multi=True, path=None, process=False, test=False, train=False)
pid: 986
Args: limit 10240
Args: epochs 1
Args: batch 160
Args: labels ['all']
Args: path /content/drive/Shareddrives/GPTJ/data/kaggle/proc-1
Args: model_name gpt2
Args: multi True
Model: name gpt2
Model: label 'all'
Model: num_labels 9
Model: using saved tokenizer...
/content/drive/Shareddrives/GPTJ/data/gpt2/all.limit-10240.chunk-32.epochs-1.batch-160/results.csv
GOOD: 1069
BAD: 14
[[150   2   0   0   0   0   0   2   0]
 [  1 246   0   0   0   0   0   0   0]
 [  0   0 294   0   0   0   0   0   0]
 [  0   0   0  47   0   0   0   0   0]
 [  0   1   0   0   3   0   0   0   0]
 [  0   0   0   0   0  75   0   0   0]
 [  1   0   1   0   0   0  37   0   0]
 [  5   0   0   0   0   0   0 117   0]
 [  0   0   0   0   0   0   0   1 100]]
{'all': {'accuracy': 0.987072945521699,
         'f1': array([0.96

# Tensorboard

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir /content/drive/Shareddrives/GPTJ/data/gpt2/7.limit-256.chunk-32.epochs-5.batch-16/logs

# Temp

In [ ]:
from statistics import mean

l1 = [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1]
print(mean(l1), round(mean(l1)))

l2 = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
print(mean(l2), round(mean(l2)))

In [ ]:
from collections import Counter

L = [1,   1, 3, 5, 6, 6, 6, 7]
W = [2.5, 2, 1, 1, 1, 1, 1, 1]

def WeightedCounter(L, W):
    c = {}
    for l, w in zip(L, W):
        if not l in c:
            c.update({l:0})
        c[l] += w
    return Counter(c)

c1 = Counter(L).most_common()[0][0]
c2 = WeightedCounter(L, W).most_common()[0][0]

print(c1)
print(c2)

In [ ]:
s1 = set(['a', 'b', 'c'])
s2 = set(['a', 'd', 'e'])
s3 = set(['b', 'c', 'e'])

s = set.intersection(s1, s2, s3)
s

set()

In [ ]:
import itertools

l = 3*[None]
l[0] = [1, 2, 3]
l[1] = [4, 5, 6]
l[2] = [7, 8, 9]

list(itertools.chain.from_iterable( [ l[i] for i in range(2) ] ))


[1, 2, 3, 4, 5, 6]

In [ ]:
l = [1, 2, 3, 4, 5, 6]


i = 1
l = l[i:] + l[:i]
l

[2, 3, 4, 5, 6, 1]